# Null Value Analysis

## Data Import

In the following code section we are importing the data from the CSV format into a panda DataFrame. The initial data format has a row per questionnaire score which can be tricky to process. Indeed, such a format doesn't let us easily see which user does not have a score for a given question/questionnaire. For that reason we first group by Client Nr, set the score names as columns and fill the missing values with NaN. 

In the new format, rows represent users and column represent questionnaire scores. We can now easily check the number of clients and the number of questionnaire scores by printing the dataframe shape. We see there are **2625 clients** and **364 dimensions** spawned from a total of **18 questionnaires**

From our feature analysis we saw that the Algemene Intake questionnaire has two versions: V 3.0 and V4.0. However they both have the same output, we will therefore merge the two.

In [231]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.layouts import column, row
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models import LinearColorMapper, ColorMapper, CategoricalColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.transform import transform

In [13]:
output_notebook()

Loading BokehJS ...

In [240]:
data = pd.read_csv("/Users/steliosrammos/Documents/Education/Maastricht/DKE-Year3/Bachelor Thesis/data/v2/client_scores_full.csv", sep=";", decimal=",")
data.rename(columns={'client_id': 'uuid'}, inplace=True)
data = data.set_index('uuid')

data = data[data['survey_date'] != '2915-10-20']
data = data[data['survey_date'] >= '2017-01-01']
data = data[data['survey_date'] <= '2018-06-30']
data['survey_date'] = pd.to_datetime(data['survey_date'],infer_datetime_format=True)
print('Max: {},\nMin {}'.format(data['survey_date'].max(), data['survey_date'].min()))

data = data.drop('survey_date', axis=1)
data['survey_name'] = data['survey_name'].replace(['Vervallen - Algemene Intake Rughuis', 'Algemene Intake V3.0', 'Vervallen - Algemene Intake'], 'Algemene Intake')
data.head()

Max: 2018-06-29 00:00:00,
Min 2017-01-02 00:00:00


,survey_name,score_name,score_value
uuid,,,
1348001455490850631,Algemene Intake,AlgIntakeOpleidingsniveauScore_Raw,3
1348001455490850631,Algemene Intake,AlgIntakeWoonsituatieScore_Raw,3
1348001455490850631,Algemene Intake,educationLevel,32
1348001455490850631,Algemene Intake,livingConditions,40
1348001455490850631,EQ-5D-5L,EQ-5D-5L-Profile,22332


In [241]:
data.shape

(459709, 3)

In [242]:
questionnaire_names = data['survey_name'].unique()
print(questionnaire_names.shape)
questionnaire_names

(28,)


array(['Algemene Intake', 'EQ-5D-5L', 'Rand 36', 'PDI', 'TSK',
       'PHODA-SeV', 'NPRS', 'CSI', 'SBT', 'QBPDS', 'PCI', 'HADS-NL',
       'BDI-II-NL', 'OQ-45.2', 'BSI', 'UCL', 'PCS', 'NDI',
       'Arbeid - Intake - Algemene vragenlijst VAR', 'CIS', 'CISS',
       'painDETECT', '4DKL', 'SCL 90', 'DAPP-BQ', 'YSQ 2.0',
       'Invuloefening Tevredenheidsenquête Het Rughuis', 'SVL concept'],
      dtype=object)

In [243]:
pivoted = data.groupby('uuid')['score_name'].value_counts().unstack().fillna(np.nan)
pivoted.shape

(2675, 385)

## Questionnaires

The following code section counts how many questionnaires are filled by X % of the patients.

We observe that we can get up to 95% of the patients by selecting the folloing 9 questionnaires: 

* Algemene Intake
* Rand 36
* PDI
* TSK
* PHODA-SeV
* NPRS
* SBT
* QBPDS
* PCI
* OQ-45.2
* BSI
* UCL
* PCS
* NDI

(CODE CELL HIDDEN)

In [244]:
# Returns the surveys that have been filled by $perc_filled$ of the clients (for at least on score)

def filled_questionnaires(df, perc_filled, filled_df):
    
    filled_scores = []
    size = df.shape[0]
    
    for column in df.columns:

        num_nulls = df[column].isna().value_counts(sort=False)[1]
        
        if num_nulls <= (size * (1-perc_filled)):
            filled_scores.append(column)
    
    filled_questionnaires = data[data['score_name'].isin(filled_scores)]['survey_name'].unique()
    num_filled_questionnaires = len(filled_questionnaires)
    
    is_filled = {}

    all_questionnaires = data['survey_name'].unique()
    
    for questionnaire in all_questionnaires:
        if questionnaire in filled_questionnaires:
            filled_df.loc[perc_filled, questionnaire] = 1
        else:
            filled_df.loc[perc_filled, questionnaire] = 0
    
    return num_filled_questionnaires, filled_df, filled_scores

In [226]:
filled_per_perc = []
percentages = np.linspace(0.05, 1, 20)

for i in range(0, len(percentages)):
    percentages[i] = round((percentages[i]), 2)

index = percentages
cells = np.zeros((len(percentages),len(questionnaire_names)))
filled_df = pd.DataFrame(cells, index=percentages, columns=questionnaire_names)
filled_df

for percentage in percentages:
    num_filled, filled_df, _ = filled_questionnaires(pivoted, percentage, filled_df)
    filled_per_perc.append(num_filled)

In [227]:
filled_df_v2 = filled_df.copy()

In [228]:
filled_df_v2

,Algemene Intake,EQ-5D-5L,Rand 36,PDI,TSK,PHODA-SeV,NPRS,CSI,SBT,QBPDS,...,zz_obsolete_BDI,4DKL,DAPP-BQ,SVL concept,PSK,YSQ 2.0,Invuloefening Tevredenheidsenquête Het Rughuis,HIT-6,SIMS,SMI
0.05,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.15,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.20,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.25,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.30,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.35,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.40,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.45,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.50,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [230]:
# here the plot :
output_file("filled_questionnaires_all_years.html")

# GRAPH 1

filled_df_v2 = filled_df.copy()
filled_df_v2.index.name = 'Percentages'
filled_df_v2.columns.name = 'Questionnaires'

df = pd.DataFrame(filled_df_v2.stack(), columns=['filled']).reset_index()
df = df.astype(str)
source = ColumnDataSource(df)

colors = ['#ffffff', '#3157aa']
mapper = LinearColorMapper(palette=colors, low=0, high=1)

x_range = list((filled_df_v2.index.astype(str).unique()))
y_range = list(filled_df_v2.columns.astype(str).unique())

g1 = figure(plot_width=1300, plot_height=600, title="Questionnaires Filled per Percentage (All Years)",
           x_range=x_range, y_range=y_range,
           toolbar_location=None, tools="", x_axis_location="above")

g1.rect(x="Percentages", y="Questionnaires", width=1, height=1, source=source,
       line_color="silver",line_width=0.2, color={'field': 'filled', 'transform': mapper}, legend='filled')

g1.xaxis.axis_label = "Percentage"
g1.yaxis.axis_label = "Questionnaire"

g1.axis.major_label_text_font_size = "5pt"
g1.axis.major_label_standoff = 0
g1.xaxis.major_label_orientation = 1.0

# GRAPH 2
hover = HoverTool(
            tooltips=[
                ("data (x,y)", "(@x, @y)")
                ])

TOOLS = [hover]

g2 = figure(plot_width=1300, 
            tools=TOOLS, 
            x_range=x_range,
            x_axis_location="above"
           )
g2.circle(x_range, filled_per_perc, size=10)

g2.yaxis.axis_label = "Num_filled"

p = column(g1, g2)
show(p)
# show(p)

In [207]:
_, filled_questionnaires_80, filled_scores_80 = filled_questionnaires(pivoted, 0.80)
filled_questionnaires_80

array(['Rand 36', 'PDI', 'NPRS', 'SBT', 'QBPDS', 'PCI', 'UCL', 'PCS',
       'NDI'], dtype=object)

In [245]:
_, filled_questionnaires_90, filled_scores_90 = filled_questionnaires(pivoted, 0.90, filled_df)
filled_questionnaires_90.columns

,Algemene Intake,EQ-5D-5L,Rand 36,PDI,TSK,PHODA-SeV,NPRS,CSI,SBT,QBPDS,...,zz_obsolete_BDI,4DKL,DAPP-BQ,SVL concept,PSK,YSQ 2.0,Invuloefening Tevredenheidsenquête Het Rughuis,HIT-6,SIMS,SMI
0.05,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.15,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.20,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.25,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.30,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.35,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.40,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.45,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.50,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [246]:
pivoted_reduced = pivoted.loc[:, filled_scores_90]
pivoted_reduced.shape

(2675, 160)

## Patients

The following code section analysis the number of patients that have filled X % of the questionnaire scores on the full set of questionnaires **and** on the reduced set of questionnaires (previously seen) 

In [247]:
# If more than $perc_filled$ % of the vlaues are missing in that row, add it to the drop_rows

def drop_rows(df, perc_filled):
    
    size = df.shape[0]
    mask = []
    num_false = 0
    
    for i in range(0, df.shape[0]):
        row = df.iloc[i, :]
        num_nulls = row.isna().value_counts(sort=False)
        
        if num_nulls.shape[0] == 2:
            if num_nulls[1] > size * (1-perc_filled):
#                 print("Row {} has {} null values".format(i, num_nulls[1]))
                mask.append(False)
                num_false += 1
            else: 
                mask.append(True)
        
        else:
            mask.append(True)
        
    num_true = size-num_false
    
    return num_true, mask
        

### Full Data Set And Reduced

In [248]:
output_file("patients_scores_filles.html")

# GRAPH 1
filled_per_perc = []
percentages = [0.2, 0.4, 0.6, 0.8, 0.85, 0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 1]

for percentage in percentages:
    
    num_true, _ = drop_rows(pivoted, percentage)
    filled_per_perc.append(num_true)

    
hover = HoverTool(
            tooltips=[
                ("index", "$index"),
                ("data (x,y)", "(@x, @y)"),
                ])

TOOLS = [hover]

s1 = figure(tools=TOOLS, title="Percentage Of Scores Filled By Patients (before reduction)")
s1.line(percentages, filled_per_perc)



# GRAPH 2
filled_per_perc = []

for percentage in percentages:
    
    num_true, _ = drop_rows(pivoted_reduced, percentage)
    filled_per_perc.append(num_true)


s2 = figure(tools=TOOLS, title="Percentage Of Scores Filled By Patients (after reduction >90%)")
s2.line(percentages, filled_per_perc)

s1.yaxis.axis_label = s2.yaxis.axis_label = "Number of Patients"
s1.xaxis.axis_label = s2.xaxis.axis_label = "Percentage"
show(row(s1,s2))

### Reduced Data Set

In [205]:
filled_per_perc = []
percentages = [0.2, 0.4, 0.6, 0.8, 0.85, 0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 1]

for percentage in percentages:
    
    num_true, _ = drop_rows(pivoted_reduced, percentage)
    filled_per_perc.append(num_true)

    
hover = HoverTool(
            tooltips=[
                ("index", "$index"),
                ("data (x,y)", "(@x, @y)"),
                ])

TOOLS = [hover]

p = figure(tools=TOOLS)
p.line(percentages, filled_per_perc)
show(p)

## Go / No-Go's

In [171]:
client_info = pd.read_csv('/Users/steliosrammos/Documents/Education/Maastricht/DKE-Year3/Bachelor Thesis/data/v2/clients_go_nogo_finished.csv', sep=';')
client_info.head()

,client_id,got_go,start_date,finished_treatment
0,3217291183496531817,1,2017-01-27,1
1,-6411020798128279438,1,2015-01-06,1
2,-4577661649179764102,1,2015-12-22,1
3,2553788127729453672,1,2015-01-05,1
4,2757733669170398576,1,2017-09-20,1


In [172]:
client_info['got_go'].value_counts()

1    4968
0    1498
Name: got_go, dtype: int64

In [189]:
pd.crosstab(client_info['got_go'], client_info['finished_treatment'], margins=True)

finished_treatment,0,1,All
got_go,,,
0,1498,0,1498
1,1044,3924,4968
All,2542,3924,6466


In [188]:
pd.crosstab(client_info['got_go'], client_info['finished_treatment'], margins=True, normalize='all')

finished_treatment,0,1,All
got_go,,,
0,0.231673,0.000000,0.231673
1,0.161460,0.606867,0.768327
All,0.393133,0.606867,1.000000
